In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install transformers

Imports

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

import numpy as np

import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

print('Using Tensorflow version:', tf.__version__)

Using Tensorflow version: 2.2.0


Loading Data

In [4]:
import pandas as pd

train_df = pd.read_csv('/content/drive/My Drive/Sentiment Analysis/Clean/train2.csv')
test_df = pd.read_csv('/content/drive/My Drive/Sentiment Analysis/Clean/test2.csv')

print (train_df.shape, test_df.shape)

(146742, 2) (60427, 2)


Data Preprocessing

In [5]:
MODEL = 'distilbert-base-multilingual-cased' # bert-base-multilingual-uncased #jplu/tf-xlm-roberta-large

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [6]:
def regular_encode(texts, tokenizer, maxlen):
    enc_di = tokenizer.batch_encode_plus(
             texts, 
             return_token_type_ids=False,
             pad_to_max_length=True,
             truncation=True,
             max_length=maxlen)
    
    return np.array(enc_di['input_ids'])

In [7]:
train_df.head(5)

,review,label
0,thanks smiling face with smiling eyes,2
1,i ordered anf black army green color but this ...,0
2,good quality size fits,3
3,excellent product quality,4
4,1 star break because i broke mine shattered my...,2


In [8]:
from tensorflow.keras.utils import to_categorical

# convert to one-hot-encoding-labels
train_labels = to_categorical(train_df['label'], num_classes=5)
train_labels

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

Split into Training and Validation set

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_df['review'].astype(str),
                                                  train_labels,
                                                  stratify=train_labels,
                                                  test_size=0.1, #10% of data
                                                  random_state=2020)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((132067,), (14675,), (132067, 5), (14675, 5))

Apply Tokenizer

In [10]:
max_len = 270

X_train = regular_encode(X_train.values, tokenizer, maxlen=max_len)
X_val = regular_encode(X_val.values, tokenizer, maxlen=max_len)
X_test = regular_encode(test_df['review'].astype(str), tokenizer, maxlen=max_len)

Fine tune model

In [11]:
def build_model(transformer, max_len=270):
    
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(5, activation='softmax')(cls_token) # 5 ratings to predict
    
    model = Model(inputs=input_word_ids, outputs=out)
    
    return model

TPU configuration

In [12]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.23.213.194:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.23.213.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.23.213.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


Training Args

In [13]:
# Configuration
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
n_steps = X_train.shape[0] // BATCH_SIZE

Generate dataset

In [14]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train, y_train))
    .repeat()
    .shuffle(1024)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_val, y_val))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(X_test)
    .batch(BATCH_SIZE)
)

Callbacks

In [ ]:
import os
from tensorflow import keras

# Prepare a directory to store all the checkpoints.
checkpoint_dir = '/content/TPU_checkpoint'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

from tensorflow import keras

callbacks = [
    #save weights every epoch
    keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir + '/Ckpt.{epoch:02d}-{loss:.2f}.h5',save_weights_only=True, save_freq='epoch')
]

Build Model

In [15]:
%%time

with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=max_len)
    #model.load_weights('/content/TPU_checkpoint/Ckpt.01-0.70.h5')
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 270)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 270, 768),)       134734080 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 5)                 3845      
Total params: 134,737,925
Trainable params: 134,737,925
Non-trainable params: 0
_________________________________________________________________
CPU times: user 20.7 s, sys: 13.7 s, total: 34.4 s
Wall time: 1min 5s


Training

In [ ]:
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    callbacks=callbacks,
    epochs=2
)

 595/1031 [================>.............] - ETA: 1:19 - loss: 1.0539 - accuracy: 0.4877

Saving weights

In [ ]:
model.save_weights("/content/weights.h5") #saving manually

Plot training Loss

In [ ]:
plt.style.use('fivethirtyeight')

# Get training and test loss histories
training_loss = train_history.history['loss']
test_loss = train_history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

Prediction

In [ ]:
pred = model.predict(test_dataset, verbose=1)

In [ ]:
# don't forget to save for ensemble
#np.save('xlm-roberta', pred)

In [ ]:
pred_sentiment = np.argmax(pred, axis=1)

print(pred_sentiment)

In [ ]:
submission = pd.DataFrame({'review_id': test_df['review_id'],
                           'rating': pred_sentiment})

Convert rating values

In [ ]:
# convert back to original rating
rating_mapper_decode = {0: 1,
                        1: 2,
                        2: 3,
                        3: 4,
                        4: 5}

In [ ]:
submission['rating'] = submission['rating'].map(rating_mapper_decode)

submission.to_csv('submission', index=False)